<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# Building a groundwater model from scratch

This notebook shows how to build a basic model from scratch using `nlmod`.

In [ ]:
# import packages
import flopy as fp
import matplotlib.pyplot as plt
import nlmod
import numpy as np
import pandas as pd

In [ ]:
# Show logging messages with level "INFO" or higher
nlmod.util.get_color_logger("INFO");

## Download MODFLOW-binaries
The first time you use nlmod you need to download the MODFLOW-excecutables. We do this with the following code:

In [ ]:
if not nlmod.util.check_presence_mfbinaries():
    nlmod.util.download_mfbinaries()

## Model parameters

Set some modelgrid parameters

In [ ]:
extent = [-500, 500, -500, 500] #extent of the model [xmin, xmax, ymin, ymax]

top = 0.0 #top level of the first model layers
botm = [-10, -15, -30] #bottom level of every model layer 

kh = [10, 0.1, 20] # horizontal conductivity per model layer
kv = [0.5 * k for k in kh] # vertical conductivity per model layer

dx = 10.0 # cell size in x direction
dy = 10.0 # cell size in y direction

### Create model dataset

Create a model dataset with the coördinates `layer`, `x` and `y`. The model data such as top, botm, kh, kv are stored in the modeldataset with the data linked to the relevant coördinates.

In [ ]:
ds = nlmod.get_ds(
    extent,
    delr=dx,
    delc=dy,
    top=top,
    botm=botm,
    kh=kh,
    kv=kv,
    model_ws="./scratch_model",
    model_name="from_scratch",
)
ds

### Set time discretisation

Add the time dimension to the model dataset. In this case we use the default settings to create a steady state model with a single time step.

In [ ]:
ds = nlmod.time.set_ds_time(ds, time=pd.Timestamp.today(), steady_state=True)
ds.time

### Start building model

Here we build the modflow package using the model dataset.

In [ ]:
sim = nlmod.sim.sim(ds)
tdis = nlmod.sim.tdis(ds, sim)
ims = nlmod.sim.ims(sim, complexity="SIMPLE")
gwf = nlmod.gwf.gwf(ds, sim)
dis = nlmod.gwf.dis(ds, gwf)
npf = nlmod.gwf.npf(ds, gwf)
ic = nlmod.gwf.ic(ds, gwf, starting_head=1.0)
oc = nlmod.gwf.oc(ds, gwf, save_head=True)

### Add wells

We add two extraction wells to the model. They have an extraction rate of respectively 100.0 and 300.0 m3/day.

In [ ]:
wells = pd.DataFrame(columns=["x", "y", "top", "botm", "Q"], index=range(2))
wells.index.name = "well no."
wells.loc[0] = 100, -50, -5, -10, -100.0
wells.loc[1] = 200, 150, -20, -30, -300.0
wells

In [ ]:
# create the well package using the DataFrame
wel = nlmod.gwf.wells.wel_from_df(wells, gwf, save_flows=True, pname='WEL')

### Add river

In [ ]:
xyriv = [
    (250, -500),
    (300, -300),
    (275, 0),
    (200, 250),
    (175, 500),
]

riv_layer = 0  # add to first layer

bed_resistance = 0.1  # days
riv_cond = dx * dy / bed_resistance  # conductance
riv_stage = 1.0  # m NAP
riv_botm = -3.0  # m NAP

riv_data = nlmod.gwf.surface_water.rivdata_from_xylist(
    gwf, xyriv, riv_layer, riv_stage, riv_cond, riv_botm
)

In [ ]:
riv = fp.mf6.ModflowGwfriv(gwf, stress_period_data={0: riv_data}, 
                           save_flows=True, pname='RIV')

### Write and run Simulation

In [ ]:
nlmod.sim.write_and_run(sim, ds)

### Load heads

In [ ]:
ds['heads'] = nlmod.gwf.get_heads_da(ds)

### Plot heads

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

pc = nlmod.plot.data_array(
    ds['heads'].sel(layer=1).isel(time=0),
    ds=ds,
    cmap="RdYlBu",
    ax=ax,
    edgecolor="k",
)
ax.axis(extent)
cbar = ax.figure.colorbar(pc, shrink=1.0)
cbar.set_label("head [m NAP]")
ax.set_title("head layer 1")
fig.tight_layout()

## Exercises

#### Exercise 1

In the plot above the heads in model layer 1 are plotted. Make 2 other plots with:
1. The heads in model layer 2 
2. The differences in heads between model layer 1 and 2

Please explain the differences in heads between these layers.

#### Exercise 2

We used the function `nlmod.gwf.get_heads_da` to obtain the modelled heads. Now use the function `nlmod.gwf.get_budget_da` to obatin the flux (budget) of the river package. This data shows how much water infiltrates from the river to the groundwater.
1. Plot the river budget of the first model layer. Explain what you see
2. Calculate the total infiltration from the river to the groundwater.
3. What other fluxes do you expect in this model? How big are they? Can you explain the differences in these fluxes?

#### Exercise 3

Add a constant recharge of 0.7 mm/day to the model using the recharge package. You can use this flopy function: `flopy.mf6.ModflowGwfrch`. Plot the heads and compare this with the previous heads. What differences do you see? 

#### Exercise 4

By setting the `time` coordinates of the model dataset `ds` you can make the model transient. Modify the function `nlmod.time.set_ds_time` to create a transient model with 30 time steps of one day. Have a look at this example model to see how to set the transient time steps: https://nlmod.readthedocs.io/en/stable/examples/03_local_grid_refinement.html

After you've done that change the well extraction rate. Start with the extraction rates as before. After 5 days set the extraction rates to zero. Plot the groundwater head over at a distance of 20, 40 and 100 meters from both extraction wells.